### Qdrant Memory

In [ ]:

class QdrantMemory:
    """
    Class responsible for embeddings governance and curation.

    This class centralizes operations related to embeddings life cycle
    ("vector memories"), including tasks such as persistence, rating,
    cleaning and validation. Ideal for systems based on RAG,
    NLP, or semantic engines.
    """
    def __init__(self, qdrant_client, embedding, collection_name=collection_name):
        self.qdrant_client = qdrant_client
        self.embedding = embedding
        self.collection_name = collection_name
        self.qdrant_db = QdrantVectorStore(client=qdrant_client, collection_name=collection_name, embedding=embedding)

    def instruct(self, question, sql, metadata=None):
        similares = self.qdrant_db.similarity_search(question, k=1)
        if similares:
            print("\n⚠️ Embedding similar já existe. Incrementando score...")
            self.increment_score(question)
            return

        vector = self.embedding.embed_query(question)
        unique_id = str(uuid.uuid4())

        point = PointStruct(
            id=unique_id,
            vector=vector,
            payload={
                "page_content": f"question: {question}\nSQL: {sql}",
                **(metadata or {}),
                "score": 1
            }
        )
        self.qdrant_client.upsert(collection_name=self.collection_name, points=[point])
        print("✅ Embedding armazenado na memoria com sucesso!")

    def increment_score(self, question):
        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=self.embedding.embed_query(question),
            limit=1
        )
        for item in result:
            point_id = item.id
            payload = item.payload or {}
            print('\nScore: ',self.list_rating(payload["score"], max_score=5))
            payload["score"] = payload.get("score", 1) + 1
            self.qdrant_client.set_payload(collection_name=self.collection_name, payload=payload, points=[point_id])

    def delete_low_score(self, limite_score):
        points_to_delete = self.qdrant_client.scroll(
            collection_name=self.collection_name,
            limit=100,
            filter=Filter(must=[FieldCondition(key="score", match=MatchValue(value=None))])
        )
        for point in points_to_delete[0]:
            score = point.payload.get("score", 0)
            if score < limite_score:
                self.qdrant_client.delete(collection_name=self.collection_name, points_selector={"points": [point.id]})

    def list_embedding_content(self):
        docs = self.qdrant_db.similarity_search("", k=100)
        for i, doc in enumerate(docs):
            print(f"\n🧬 Embedding {i+1}:")
            print(doc.page_content)


    def seach_embedding(self, consulta, filtro):
        filtered_docs = self.qdrant_db.similarity_search(consulta, k=3, filter=filtro)
        for i, doc in enumerate(filtered_docs):
            print(f"\n🧬 Embedding filtrado {i+1}:")
            print(doc.page_content)

    def list_rating(self, score, max_score=5):
        if score:
            min_rating = min(score, max_score)
            score_str = '⭐' * score + '✩' * (max_score - min_rating)
            return score_str
        else:
            return '✩✩✩✩✩'

    def list_scored_point(self, question):

        result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=self.embedding.embed_query(question),
            limit=1
        )
        print("📦 Payload:",result)
